In [ ]:
from ml_carbucks import DATA_DIR
from ml_carbucks.adapters.UltralyticsAdapter import YoloUltralyticsAdapter, RtdetrUltralyticsAdapter
from ml_carbucks.adapters.FasterRcnnAdapter import FasterRcnnAdapter
from ml_carbucks.adapters.EfficientDetAdapter import EfficientDetAdapter
from ml_carbucks.adapters.BaseDetectionAdapter import ADAPTER_DATASETS

cardd_dataset_train: ADAPTER_DATASETS = [(
    DATA_DIR / "car_dd" / "images" / "train",
    DATA_DIR / "car_dd" / "annotations" /"instances_train_curated.json",
)]
cardd_dataset_val: ADAPTER_DATASETS = [(
    DATA_DIR / "car_dd" / "images" / "val",
    DATA_DIR / "car_dd" / "annotations" /"instances_val_curated.json",
)]

v, dec = "8", "checkpoint_cleanup"


In [ ]:

from typing import List, Optional

from ml_carbucks.adapters.BaseDetectionAdapter import BaseDetectionAdapter

params = {
    "img_size": 384,
    "batch_size": 8,
    "epochs": 5,
    "verbose": True,
}

override_epochs: Optional[int] = 5

adapters: List[BaseDetectionAdapter] = [
    YoloUltralyticsAdapter().set_params({
        "img_size": 384,
        "batch_size": 8,
        "epochs": override_epochs or 18,
        "lr": 0.00012520173287645337,
        "momentum": 0.7751745737454412,
        "weight_decay": 1.2339139608460587e-05,
        "optimizer": "Adam"
    }),
    RtdetrUltralyticsAdapter().set_params( {
        "img_size": 384,
        "batch_size": 16,
        "epochs": override_epochs or 28,
        "lr": 0.0006706879778698832,
        "momentum": 0.5574030731076479,
        "weight_decay": 0.00029530220750148133,
        "optimizer": "Adam"
    }),
    FasterRcnnAdapter().set_params({
        "img_size": 384,
        "batch_size": 16,
        "epochs": override_epochs or 22,
        "lr_head": 0.0001760393933421673,
        "weight_decay_head": 1.6330563888408336e-05,
        "optimizer": "AdamW",

    }),
    EfficientDetAdapter().set_params({
        "img_size": 384,
        "batch_size": 32,
        "epochs": override_epochs or 15,
        "optimizer": "adam",
        "lr": 0.0002740546809702111,
        "weight_decay": 8.588886298464197e-05,
        "loader": "inbuild"
    }),
]
results = []

for adapter in adapters:
    adapter.set_params(params).setup()
    adapter.fit(cardd_dataset_train)
    res = adapter.evaluate(cardd_dataset_val)

    results.append(res)

print("Evaluation results:")
for res in results:
    print(res)


In [ ]:

for iii in range(len(adapters)):
    adapters[iii].save(f"/home/bachelor/ml-carbucks/results/pickle{v}", prefix=f"{adapters[iii].__class__.__name__}_")

In [ ]:
from typing import List

from ml_carbucks.adapters.BaseDetectionAdapter import BaseDetectionAdapter


loaded_adapters: List[BaseDetectionAdapter] = [
    YoloUltralyticsAdapter(weights=f"/home/bachelor/ml-carbucks/results/pickle{v}/YoloUltralyticsAdapter_model.pkl"),
    RtdetrUltralyticsAdapter(weights=f"/home/bachelor/ml-carbucks/results/pickle{v}/RtdetrUltralyticsAdapter_model.pkl"),
    FasterRcnnAdapter(weights=f"/home/bachelor/ml-carbucks/results/pickle{v}/FasterRcnnAdapter_model.pkl"),
    EfficientDetAdapter(weights=f"/home/bachelor/ml-carbucks/results/pickle{v}/EfficientDetAdapter_model.pkl"),
]


In [ ]:
lesults = []
for jjj in range(len(loaded_adapters)):
    try:
        loaded_adapters[jjj].setup()
        les = loaded_adapters[jjj].evaluate(cardd_dataset_val)
        lesults.append(les)
    except Exception:
        lesults.append("Evaluation failed")

print("Evaluation results of loaded adapters:")
for les in lesults:
    print(les)

In [ ]:
print("Comparing results")
metric = "map_50"

for original, loaded in zip(results, lesults):
    print("Original:", original[metric])
    print("Loaded:  ", loaded[metric])

NOTES:
- X yolo works fine 
- X rtdetr works fine
- X fasterRCNN works fine
- X efficientDet works fine